In [177]:
#Loading the necessary packages
import pandas as pd
import re
from datetime import datetime,timedelta,date
from dateutil.relativedelta import *
from warnings import filterwarnings
filterwarnings("ignore")
##Dataframe Global variables
pd.set_option('display.max_columns',40)
pd.set_option('max_colwidth',50)

In [178]:
##Conditions
## Data Should be present till lastmonth
## Input data of the months considered should be complete i.e till end of every month else will be eliminated.

In [179]:
def read_file(filepath,filetype,encoding='ISO-8859-1'):
    """
    This function reads txt and csv files and returns the data
    """
    if(filetype=="txt" or "csv"):
        data = pd.read_csv(filepath,encoding=encoding,low_memory=False)    
    else:
        raise Exception("Kindly Check! Only txt or csv filetype allowed")
    return data

In [180]:
def read_file_chunks(filepath,filetype,chunk_size,encoding='ISO-8859-1'):
    """
    This function reads data from the file in chunks
    """
    if(filetype=="txt" or "csv"):
        chunk_data = pd.read_csv(filepath,encoding=encoding,low_memory=False,chunksize=chunk_size)
        for chunks in range(0,999):
            if(chunks==0):
                card_txn_data = next(chunk_data)
            else:
                try:
                    card_txn_data = card_txn_data.append(next(chunk_data))
                except StopIteration:
                    break
    else:
        raise Exception("Kindly Check! Only txt or csv filetype allowed")
    return card_txn_data  

In [181]:
def clean_filter_cardname(name,rm_space=""):
    """
    This function is used to get cleaned card acceptor name
    """
    name = name.lower()
    #remove numbers special characters and concatenate
    name = re.sub("[^a-zA-Z\s]","",name)
    ## removes space inbetween words
    if(rm_space == 'X'):
        name = re.sub("[^a-zA-Z]","",name)
    return name

In [182]:
def tag_auto_txn(row):
    if(row.merchant_cats_lowercase.find('taxicabs')!=-1):
        if((str(row.clean_txn_name).find('uber')!=-1)&(str(row.clean_txn_name).find('eats')==-1)):
            value = 'uber'
        elif(str(row.clean_txn_name).find('lyft')!=-1):
            value = 'lyft'
        else:
            value =''  
    else:
        value = ''    
    return value

In [183]:
def condition_check(cust,data,min_limit,txn_hist_date):
    """
    This function checks if provided min limit of txns is met in past 3 months of txn data of a customer.
    if condition is satisfied it returns 1 else 0
    """
    mon_1 = txn_hist_date-timedelta(31)
    mon_2 = txn_hist_date-timedelta(61)
    mon_3 = txn_hist_date-timedelta(91)
    pastmonth_1 = str(format(mon_1.month,'02'))+'_'+str(mon_1.year)
    pastmonth_2 = str(format(mon_2.month,'02'))+'_'+str(mon_2.year)
    pastmonth_3 = str(format(mon_3.month,'02'))+'_'+str(mon_3.year)
    #print(records,records_min_txns)
    if(len(data.loc[(data['MO_YR'].isin([pastmonth_1,pastmonth_2,pastmonth_3]))&(data.txn_cnt_month>=min_limit),'MO_YR'])==3):
        return 1
    else:
        return 0

## Read the data

In [184]:
filepath = r'C:\Users\ujjwa\Downloads\Grow Backup\Grow\data\Customers_10000\Customers_10000.csv'
filetype = "csv"
encoding = "ISO-8859-1"
chunksize = 350000
card_txn_data = read_file_chunks(filepath,filetype,chunksize)
#card_txn_data = read_file(filepath,filetype)
if(len(card_txn_data)==0):
    raise Exception("Data not loaded Kindly check!format of the data file")

In [186]:
required_cols = ['ArtificialAccountKey', 'POSTAMOUNT','CARDACCEPTORCITY',
                 'CARDACCEPTORCOUNTRY', 'CARDACCEPTORNAME', 'CARDACCEPTORSTATE',
                 'CARDACCEPTORSTREET', 'LOCALTRANDATE', 'LOCALTRANTIME',
                 'MerchantCategory','MISCDATA3','OURACCTCODE1', 'OURCARDTYPE', 'OURSYSTEMDATE', 'OURTRANCODE',
                 'RECURRINGFLAG']
card_txn_data = card_txn_data[required_cols]

In [189]:
##Categories related to cab services.
rental_tags = ['taxicabs','limousines']
##Initialization of variable
auto_txn_categories = []

In [190]:
##Required columns
card_txn_data['LOCALTRANDATE'] = pd.to_datetime(card_txn_data.LOCALTRANDATE)
card_txn_data['MONTH'] = card_txn_data['LOCALTRANDATE'].apply(lambda x:format(x.month,'02'))
card_txn_data['YEAR'] = card_txn_data['LOCALTRANDATE'].apply(lambda x:x.year)
card_txn_data['MO_YR'] = card_txn_data['MONTH'].astype('str')+'_'+card_txn_data['YEAR'].astype('str')
card_txn_data['DATE'] = card_txn_data['LOCALTRANDATE'].apply(lambda x:datetime.strftime(x,'%Y-%m-%d'))

In [191]:
##Get clean card acceptor names
card_txn_data['clean_txn_name'] = card_txn_data['CARDACCEPTORNAME'].apply(lambda x:clean_filter_cardname(x))
##column with merchant category in lower case
card_txn_data['merchant_cats_lowercase'] = card_txn_data['MerchantCategory'].apply(lambda x:str(x).lower())

In [193]:
## Fetching Categories related to cabs (uber and lyft)
merchant_cats = card_txn_data.merchant_cats_lowercase.unique()
for cats in merchant_cats:
    count = 0
    for word in rental_tags:
        if(str(cats).find(word)!=-1):
            count = count+1
    ##If more than one tag is present in category name
    if(count>1):
        auto_txn_categories.append(cats)
##Raise an exception if no data is present
if(len(auto_txn_categories)==0):
    raise Exception("Kindly check! No cab or rental related categories data is present")

In [195]:
##Last date of recorded txn
txn_last_date = card_txn_data.LOCALTRANDATE.max()
#changing the end date to the first day of the last month, as the month is incomplete.
end_date = txn_last_date.replace(day=1)
## Working on data related only to auto transactions and before the last full month transaction.
auto_txn_data = card_txn_data.loc[(card_txn_data.merchant_cats_lowercase.isin(auto_txn_categories))&(card_txn_data.LOCALTRANDATE<end_date),]
auto_txn_data.reset_index(inplace=True,drop=True)
##Check if auto_txn_data is not empty
if(len(auto_txn_data)==0):
    raise Exception("Kindly check! No data related to auto transactions")

In [196]:
auto_txn_data['auto_txn_tag'] = auto_txn_data.apply(tag_auto_txn,axis=1)

In [198]:
##Remove other transactions related to uber eats and other cab services
auto_final_data = auto_txn_data.loc[auto_txn_data.auto_txn_tag!='',]

In [199]:
##GroupBy and count values per month
auto_data_grouped = auto_final_data.groupby(by=['ArtificialAccountKey','auto_txn_tag','MO_YR'],as_index=False)['POSTAMOUNT'].agg({'monthly_amt_spent':'sum', 'txn_cnt_month':'count'})

In [201]:
##List of Customers to be analyzed
customers_list = auto_data_grouped.ArtificialAccountKey.unique().tolist()

## Main Logic

In [203]:
##Main Logic for each customer to capture the customers making frequent cab transactions.
uber_cust,lyft_cust = [],[]
for cust_key in customers_list:
    uber_check,lyft_check = 0,0
    uber_data = auto_data_grouped.loc[(auto_data_grouped.ArtificialAccountKey==cust_key)&(auto_data_grouped.auto_txn_tag =='uber'),]
    lyft_data = auto_data_grouped.loc[(auto_data_grouped.ArtificialAccountKey==cust_key)&(auto_data_grouped.auto_txn_tag =='lyft'),]
    # More than 2 months data should be present then goes into condition check
    if(len(uber_data)>2):
        #check for minimum limit of transaction required, can be changed as per business need.
        uber_check = condition_check(cust_key,uber_data,min_limit=15,txn_hist_date=txn_last_date)
        if(uber_check == 1):
            uber_cust.append(cust_key)
    if(len(lyft_data)>2):
        #check for minimum limit of transaction required, can be changed as per business need.
        lyft_check = condition_check(cust_key,lyft_data,min_limit=5,txn_hist_date=txn_last_date)
        if(lyft_check == 1):
            lyft_cust.append(cust_key)

In [204]:
##Group the customer related transaction data
uber_grouped = auto_data_grouped.loc[(auto_data_grouped.ArtificialAccountKey.isin(uber_cust))&(auto_data_grouped.auto_txn_tag =='uber')]
lyft_grouped = auto_data_grouped.loc[(auto_data_grouped.ArtificialAccountKey.isin(lyft_cust))&(auto_data_grouped.auto_txn_tag =='lyft')]

In [205]:
## Pivot table output for each customer and summary of their cab transactions.
uber_pivot = uber_grouped.pivot_table(index=['ArtificialAccountKey','auto_txn_tag'],columns='MO_YR',values='txn_cnt_month',fill_value=0)
uber_pivot.reset_index(inplace=True)
lyft_pivot = lyft_grouped.pivot_table(index=['ArtificialAccountKey','auto_txn_tag'],columns='MO_YR',values='txn_cnt_month',fill_value=0)
lyft_pivot.reset_index(inplace=True)

## Output

In [206]:
## Final Output - Dataframe uber_pivot and lyft_pivot contains transactions summary for all the customers and can be targeted for special offers.

In [ ]:
uber_pivot

In [ ]:
lyft_pivot